In [20]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
import calendar
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [5]:
#Define presto credentials
presto_host = 'bi-presto.serving.data.production.internal'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [6]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, customer_location_latitude, 
pickup_location_hex_8, pickup_location_latitude, pickup_location_longitude, drop_location_hex_8, drop_location_latitude, drop_location_longitude,
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason
from orders.order_logs_snapshot
where yyyymmdd >= '20230306'
and yyyymmdd <= '20230310'
and city_name in ('Chennai')
and service_obj_service_name = 'Link'
and hour in ('08', '09', '10', '11', '17', '18', '19', '20', '21')
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [21]:
#Create a copy of the df retrived from presto
df = df_copy.copy()
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason
0,6405e7641215c13df1c82ffc,NaN,88618c4f23fffff,13.046065,80.232277,Chennai,88618c4f23fffff,13.046065,88618c4f23fffff,13.046019,80.232299,88618c4d05fffff,13.084332,80.212763,80.232277,184516,18,1845,Link,18:59,1,20230306,customerCancelled,order cancelled before rider accepted
1,6405e78460804939c421b78f,1.509,88618c4a97fffff,13.025041,80.274681,Chennai,88618c48d9fffff,13.044349,88618c48d9fffff,13.044310,80.269089,88618c4a97fffff,13.025041,80.274681,80.268959,184548,18,1845,Link,18:59,1,20230306,dropped,None
2,6405e7b0ce0f880cb87f86cd,0.448,88618c4e15fffff,12.974972,80.186012,Chennai,88618c4e33fffff,12.987517,88618c4e33fffff,12.987517,80.176434,88618c4e15fffff,12.974972,80.186012,80.176437,184632,18,1845,Link,18:59,1,20230306,dropped,None
3,6405e7fc2030b56c339c9b73,0.815,88618c4c2dfffff,13.048095,80.192299,Chennai,88618c4f33fffff,13.044246,88618c4f33fffff,13.044189,80.212021,88618c4c2dfffff,13.048095,80.192299,80.211983,184748,18,1845,Link,18:59,1,20230306,dropped,None
4,6405e866d36fd76622783406,1.055,88618c4f07fffff,13.033194,80.230537,Chennai,88618c4f07fffff,13.033194,88618c4f07fffff,13.033197,80.230550,88618c4f21fffff,13.046565,80.238773,80.230537,184934,18,1845,Link,18:59,1,20230306,customerCancelled,Change of plans


In [22]:
#Make columns for second and minute and date
df['second'] = df['hhmmss'].apply(lambda x: x[4:])
df['minute'] = df['hhmmss'].apply(lambda x: x[2:4])
df['date'] = df['yyyymmdd'].apply(lambda x: x[6:])
df['count'] = 1
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count
0,6405e7641215c13df1c82ffc,NaN,88618c4f23fffff,13.046065,80.232277,Chennai,88618c4f23fffff,13.046065,88618c4f23fffff,13.046019,80.232299,88618c4d05fffff,13.084332,80.212763,80.232277,184516,18,1845,Link,18:59,1,20230306,customerCancelled,order cancelled before rider accepted,16,45,06,1
1,6405e78460804939c421b78f,1.509,88618c4a97fffff,13.025041,80.274681,Chennai,88618c48d9fffff,13.044349,88618c48d9fffff,13.044310,80.269089,88618c4a97fffff,13.025041,80.274681,80.268959,184548,18,1845,Link,18:59,1,20230306,dropped,None,48,45,06,1
2,6405e7b0ce0f880cb87f86cd,0.448,88618c4e15fffff,12.974972,80.186012,Chennai,88618c4e33fffff,12.987517,88618c4e33fffff,12.987517,80.176434,88618c4e15fffff,12.974972,80.186012,80.176437,184632,18,1845,Link,18:59,1,20230306,dropped,None,32,46,06,1
3,6405e7fc2030b56c339c9b73,0.815,88618c4c2dfffff,13.048095,80.192299,Chennai,88618c4f33fffff,13.044246,88618c4f33fffff,13.044189,80.212021,88618c4c2dfffff,13.048095,80.192299,80.211983,184748,18,1845,Link,18:59,1,20230306,dropped,None,48,47,06,1
4,6405e866d36fd76622783406,1.055,88618c4f07fffff,13.033194,80.230537,Chennai,88618c4f07fffff,13.033194,88618c4f07fffff,13.033197,80.230550,88618c4f21fffff,13.046565,80.238773,80.230537,184934,18,1845,Link,18:59,1,20230306,customerCancelled,Change of plans,34,49,06,1


In [9]:
#Load data from cluster table
q = """
select * from datasets.city_cluster_hex
where city in ('Chennai')
and resolution = 8
"""
#Load data into pandas table
df_cluster_copy = pd.read_sql(q, connection)

In [23]:
cluster_mapping = df_cluster_copy.copy()
cluster_mapping = cluster_mapping.rename(columns = {'hex_id':'pickup_location_hex_8'})
df = df.merge(cluster_mapping[['pickup_location_hex_8', 'cluster']], on = 'pickup_location_hex_8', how = 'left')
cluster_mapping = cluster_mapping.rename(columns = {'pickup_location_hex_8':'drop_location_hex_8'})
df = df.rename(columns = {'cluster':'pickup_cluster'})
df = df.merge(cluster_mapping[['drop_location_hex_8', 'cluster']], on = 'drop_location_hex_8', how = 'left')
df = df.rename(columns = {'cluster':'drop_cluster'})
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count,pickup_cluster,drop_cluster
0,6405e7641215c13df1c82ffc,NaN,88618c4f23fffff,13.046065,80.232277,Chennai,88618c4f23fffff,13.046065,88618c4f23fffff,13.046019,80.232299,88618c4d05fffff,13.084332,80.212763,80.232277,184516,18,1845,Link,18:59,1,20230306,customerCancelled,order cancelled before rider accepted,16,45,06,1,T Nagar,Anna Nagar
1,6405e78460804939c421b78f,1.509,88618c4a97fffff,13.025041,80.274681,Chennai,88618c48d9fffff,13.044349,88618c48d9fffff,13.044310,80.269089,88618c4a97fffff,13.025041,80.274681,80.268959,184548,18,1845,Link,18:59,1,20230306,dropped,None,48,45,06,1,Teynampet,Mylapore
2,6405e7b0ce0f880cb87f86cd,0.448,88618c4e15fffff,12.974972,80.186012,Chennai,88618c4e33fffff,12.987517,88618c4e33fffff,12.987517,80.176434,88618c4e15fffff,12.974972,80.186012,80.176437,184632,18,1845,Link,18:59,1,20230306,dropped,None,32,46,06,1,Meenambakkam,Meenambakkam
3,6405e7fc2030b56c339c9b73,0.815,88618c4c2dfffff,13.048095,80.192299,Chennai,88618c4f33fffff,13.044246,88618c4f33fffff,13.044189,80.212021,88618c4c2dfffff,13.048095,80.192299,80.211983,184748,18,1845,Link,18:59,1,20230306,dropped,None,48,47,06,1,Ashok Nagar Chennai,Alwarthirunagar
4,6405e866d36fd76622783406,1.055,88618c4f07fffff,13.033194,80.230537,Chennai,88618c4f07fffff,13.033194,88618c4f07fffff,13.033197,80.230550,88618c4f21fffff,13.046565,80.238773,80.230537,184934,18,1845,Link,18:59,1,20230306,customerCancelled,Change of plans,34,49,06,1,T Nagar,T Nagar


In [24]:
#Filter the df on morning and evening peak hours
df_morning = df[df['hour'].isin(['08', '09', '10', '11'])]
df_evening = df[df['hour'].isin(['17', '18', '19', '20', '21'])]

#Groupby on hex and quarter hour to get the number of orders coming in every hex
df_morning_grouped = df_morning.groupby(['yyyymmdd', 'hour', 'pickup_location_hex_8'])['order_id'].nunique().reset_index()
df_evening_grouped = df_evening.groupby(['yyyymmdd', 'hour', 'pickup_location_hex_8'])['order_id'].nunique().reset_index()
df_morning_grouped = df_morning_grouped.rename(columns = {'order_id':'num_of_orders'})
df_evening_grouped = df_evening_grouped.rename(columns = {'order_id':'num_of_orders'})
df_evening_grouped.head()

,yyyymmdd,hour,pickup_location_hex_8,num_of_orders
0,20230306,17,88618c4081fffff,3
1,20230306,17,88618c4083fffff,4
2,20230306,17,88618c4085fffff,2
3,20230306,17,88618c4087fffff,2
4,20230306,17,88618c408bfffff,5


In [25]:
df_morning = df_morning.merge(df_morning_grouped, on = ['yyyymmdd', 'hour', 'pickup_location_hex_8'], how = 'left')
df_evening = df_evening.merge(df_evening_grouped, on = ['yyyymmdd', 'hour', 'pickup_location_hex_8'], how = 'left')

In [26]:
df_morning['weekday_name'] = df_morning['weekday'].apply(lambda x: calendar.day_name[(x-1)])
df_evening['weekday_name'] = df_evening['weekday'].apply(lambda x: calendar.day_name[(x-1)])

In [14]:
#Query data for mismatch

q_online_captains = """
select * from pricing.cluster_temporal_hotness
where service_detail_id = '5bed473f1278885df4ea9d57'
"""


#Load data into pandas table
df_mismatch_copy = pd.read_sql(q_online_captains, connection)

In [27]:
#View the dataset
df_mismatch = df_mismatch_copy.copy()
df_mismatch.head()

,hotness_category,pickupclusters,week_period,time_period,temporal_id,total_footfall,total_revenue,hex_count,total_revenue_normalized,normalized_revenue_contribution_percentage,cum_revenue_contribution_percentage,revenue_contribution_percentile_rank,temporal_stress,temporal_mismatch,spatial_stress,spatial_mismatch,cluster_hotness_score,run_date,strategy_version,service_detail_id
0,cold,Alwarpet,Tuesday,1100,Tuesday-1100,0,0.0,0,0.0,0.0,0.0,0.0,0.377248,0.262952,0.328037,0.198446,0.291671,20230314,HOT_COLD_3_0,5bed473f1278885df4ea9d57
1,medium,Alwarpet,Tuesday,2200,Tuesday-2200,0,0.0,0,0.0,0.0,0.0,0.0,0.437693,0.566167,0.349223,0.394068,0.436788,20230314,HOT_COLD_3_0,5bed473f1278885df4ea9d57
2,medium,Alwarpet,Tuesday,1600,Tuesday-1600,0,0.0,0,0.0,0.0,0.0,0.0,0.399439,0.540735,0.337924,0.372175,0.412568,20230314,HOT_COLD_3_0,5bed473f1278885df4ea9d57
3,medium,Alwarpet,Tuesday,1700,Tuesday-1700,0,0.0,0,0.0,0.0,0.0,0.0,0.463161,0.551804,0.350282,0.409251,0.443625,20230314,HOT_COLD_3_0,5bed473f1278885df4ea9d57
4,medium,Alwarpet,Tuesday,1500,Tuesday-1500,0,0.0,0,0.0,0.0,0.0,0.0,0.365197,0.497242,0.297316,0.345339,0.376274,20230314,HOT_COLD_3_0,5bed473f1278885df4ea9d57


In [28]:
df_mismatch = df_mismatch[df_mismatch['run_date'] == df_mismatch['run_date'].max()]

In [29]:
#rename column
df_mismatch = df_mismatch.rename(columns = {'week_period':'weekday_name', 'pickupclusters':'pickup_cluster'})
df_mismatch['hour'] = df_mismatch['time_period'].apply(lambda x: x[:2])
df_mismatch.head()

,hotness_category,pickup_cluster,weekday_name,time_period,temporal_id,total_footfall,total_revenue,hex_count,total_revenue_normalized,normalized_revenue_contribution_percentage,cum_revenue_contribution_percentage,revenue_contribution_percentile_rank,temporal_stress,temporal_mismatch,spatial_stress,spatial_mismatch,cluster_hotness_score,run_date,strategy_version,service_detail_id,hour
0,cold,Alwarpet,Tuesday,1100,Tuesday-1100,0,0.0,0,0.0,0.0,0.0,0.0,0.377248,0.262952,0.328037,0.198446,0.291671,20230314,HOT_COLD_3_0,5bed473f1278885df4ea9d57,11
1,medium,Alwarpet,Tuesday,2200,Tuesday-2200,0,0.0,0,0.0,0.0,0.0,0.0,0.437693,0.566167,0.349223,0.394068,0.436788,20230314,HOT_COLD_3_0,5bed473f1278885df4ea9d57,22
2,medium,Alwarpet,Tuesday,1600,Tuesday-1600,0,0.0,0,0.0,0.0,0.0,0.0,0.399439,0.540735,0.337924,0.372175,0.412568,20230314,HOT_COLD_3_0,5bed473f1278885df4ea9d57,16
3,medium,Alwarpet,Tuesday,1700,Tuesday-1700,0,0.0,0,0.0,0.0,0.0,0.0,0.463161,0.551804,0.350282,0.409251,0.443625,20230314,HOT_COLD_3_0,5bed473f1278885df4ea9d57,17
4,medium,Alwarpet,Tuesday,1500,Tuesday-1500,0,0.0,0,0.0,0.0,0.0,0.0,0.365197,0.497242,0.297316,0.345339,0.376274,20230314,HOT_COLD_3_0,5bed473f1278885df4ea9d57,15


In [30]:
df_morning = df_morning.merge(df_mismatch[['weekday_name', 'hour', 'pickup_cluster', 'cluster_hotness_score']],
                               on = ['weekday_name', 'hour', 'pickup_cluster'], how = 'left')
df_evening = df_evening.merge(df_mismatch[['weekday_name', 'hour', 'pickup_cluster', 'cluster_hotness_score']],
                               on = ['weekday_name', 'hour', 'pickup_cluster'], how = 'left')
df_evening.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count,pickup_cluster,drop_cluster,num_of_orders,weekday_name,cluster_hotness_score
0,6405e7641215c13df1c82ffc,NaN,88618c4f23fffff,13.046065,80.232277,Chennai,88618c4f23fffff,13.046065,88618c4f23fffff,13.046019,80.232299,88618c4d05fffff,13.084332,80.212763,80.232277,184516,18,1845,Link,18:59,1,20230306,customerCancelled,order cancelled before rider accepted,16,45,06,1,T Nagar,Anna Nagar,25,Monday,0.774329
1,6405e78460804939c421b78f,1.509,88618c4a97fffff,13.025041,80.274681,Chennai,88618c48d9fffff,13.044349,88618c48d9fffff,13.044310,80.269089,88618c4a97fffff,13.025041,80.274681,80.268959,184548,18,1845,Link,18:59,1,20230306,dropped,None,48,45,06,1,Teynampet,Mylapore,36,Monday,0.858767
2,6405e7b0ce0f880cb87f86cd,0.448,88618c4e15fffff,12.974972,80.186012,Chennai,88618c4e33fffff,12.987517,88618c4e33fffff,12.987517,80.176434,88618c4e15fffff,12.974972,80.186012,80.176437,184632,18,1845,Link,18:59,1,20230306,dropped,None,32,46,06,1,Meenambakkam,Meenambakkam,16,Monday,0.378562
3,6405e7fc2030b56c339c9b73,0.815,88618c4c2dfffff,13.048095,80.192299,Chennai,88618c4f33fffff,13.044246,88618c4f33fffff,13.044189,80.212021,88618c4c2dfffff,13.048095,80.192299,80.211983,184748,18,1845,Link,18:59,1,20230306,dropped,None,48,47,06,1,Ashok Nagar Chennai,Alwarthirunagar,43,Monday,0.563607
4,6405e866d36fd76622783406,1.055,88618c4f07fffff,13.033194,80.230537,Chennai,88618c4f07fffff,13.033194,88618c4f07fffff,13.033197,80.230550,88618c4f21fffff,13.046565,80.238773,80.230537,184934,18,1845,Link,18:59,1,20230306,customerCancelled,Change of plans,34,49,06,1,T Nagar,T Nagar,31,Monday,0.774329


In [31]:
df_morning_filter = df_morning[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'weekday_name', 'cluster_hotness_score']]
df_evening_filter = df_evening[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'weekday_name', 'cluster_hotness_score']]

In [32]:
df_evening_filter.to_csv('chennai_demand_supply_evening_vw2.csv', index = False)
df_morning_filter.to_csv('chennai_demand_supply_morning_vw2.csv', index = False)

In [17]:
#Query data for online captains

q_online_captains = """
select
        yyyymmdd, substr(hhmm, 1, 2) as hour, cluster,
        count(distinct captain_id) as Online_Captains
from 
(select 
        a.*, city, cluster
        
from 
        hive.datasets.supplycursory_history a 
        left join datasets.city_cluster_hex b 
        on a.location = b.hex_id and a.resolution = cast(b.resolution as varchar)
where 
        yyyymmdd >= '20230227'
        and yyyymmdd <= '20230302'
        and city in ('Chennai')
        and a.resolution = '8' 
        and service in ('Link')
        and substr(hhmm, 1, 2) in ('17', '18', '19', '20', '21')
        and location in ('88618c4f07fffff', '88618c4f05fffff', '88618c48d1fffff', '88618c4ab3fffff', '88618c4f29fffff', '88618c4f65fffff', '88618c48d3fffff',
 '88618c4f61fffff', '88618c4f3dfffff', '88618c4f2bfffff', '88618c4f2dfffff', '88618c4f25fffff', '88618c4f35fffff', '88618c4abbfffff', '88618c48ddfffff',
   '88618c4f63fffff', '88618c4f69fffff', '88618c4d49fffff', '88618c4d4dfffff', '88618c48d5fffff', '88618c4d41fffff', '88618c4f6bfffff', '88618c48dbfffff',
     '88618c4f67fffff', '88618c4899fffff', '88618c4ab7fffff', '88618c4f01fffff', '88618c4f47fffff', '88618c4f27fffff', '88618c4f03fffff', '88618c48c3fffff',
       '88618c4f39fffff', '88618c48d9fffff', '88618c4f6dfffff', '88618c4f45fffff', '88618c4ab1fffff', '88618c48d7fffff', '88618c4f21fffff', '88618c4a97fffff',
         '88618c4f23fffff', '88618c4f4dfffff', '88618c4f0dfffff', '88618c489bfffff', '88618c48cbfffff', '88618c4f15fffff', '88618c4a91fffff', '88618c4f09fffff',
           '88618c4a93fffff', '88618c4f0bfffff')
        and cast(status AS varchar) IN ('2')
)
group by 1,2,3
"""

#Load data into pandas table
df_online_captains_copy = pd.read_sql(q_online_captains, connection)

In [18]:
#Make a copy of the dataset and view it
df_online_captains = df_online_captains_copy.copy()
df_online_captains.head()

,yyyymmdd,hour,cluster,Online_Captains
0,20230301,17,Nungambakkam,237
1,20230228,17,Nungambakkam,231
2,20230301,21,Teynampet,175
3,20230228,20,T Nagar,322
4,20230301,18,Kotturpuram,66


In [19]:
df_online_captains.groupby(['yyyymmdd', 'hour'])['Online_Captains'].sum().reset_index()

,yyyymmdd,hour,Online_Captains
0,20230227,17,1338
1,20230227,18,1595
2,20230227,19,1795
3,20230227,20,1689
4,20230227,21,1399
5,20230228,17,1257
6,20230228,18,1519
7,20230228,19,1787
8,20230228,20,1624
9,20230228,21,1371


In [20]:
#Rename cluster column
df_online_captains = df_online_captains.rename(columns = {'cluster':'pickup_cluster'})
df_online_captains.head()

,yyyymmdd,hour,pickup_cluster,Online_Captains
0,20230301,17,Nungambakkam,237
1,20230228,17,Nungambakkam,231
2,20230301,21,Teynampet,175
3,20230228,20,T Nagar,322
4,20230301,18,Kotturpuram,66


In [21]:
#Merge online captains with demand data
df_morning_grouped = df_morning_grouped.merge(df_online_captains, on = ['yyyymmdd', 'hour', 'pickup_cluster'], how = 'left')
df_evening_grouped = df_evening_grouped.merge(df_online_captains, on = ['yyyymmdd', 'hour', 'pickup_cluster'], how = 'left')
df_morning_grouped = df_morning_grouped.rename(columns = {'order_id':'num_of_orders'})
df_evening_grouped = df_evening_grouped.rename(columns = {'order_id':'num_of_orders'})
df_morning_grouped['demand_to_supply_ratio'] = round(df_morning_grouped['num_of_orders']/df_morning_grouped['Online_Captains'], 4)
df_evening_grouped['demand_to_supply_ratio'] = round(df_evening_grouped['num_of_orders']/df_evening_grouped['Online_Captains'], 4)

df_morning_grouped = df_morning_grouped.sort_values(by = 'demand_to_supply_ratio', ascending = False)
df_evening_grouped = df_evening_grouped.sort_values(by = 'demand_to_supply_ratio', ascending = False)
df_evening_grouped.head()

,yyyymmdd,hour,pickup_cluster,num_of_orders,Online_Captains,demand_to_supply_ratio
169,20230302,18,Thousand Lights Metro,92,43,2.1395
119,20230301,18,Thousand Lights Metro,87,50,1.7400
164,20230302,18,Nungambakkam,392,235,1.6681
100,20230301,17,Alwarpet,71,44,1.6136
110,20230301,18,Alwarpet,118,76,1.5526


In [27]:
df_evening_grouped.groupby(['hour'])['demand_to_supply_ratio'].mean()

hour
17    0.918695
18    1.022725
19    0.714440
20    0.515355
21    0.482165
Name: demand_to_supply_ratio, dtype: float64

In [22]:
df_morning = df_morning.merge(df_morning_grouped, on = ['yyyymmdd', 'pickup_cluster', 'hour'], how = 'left')
df_evening = df_evening.merge(df_evening_grouped, on = ['yyyymmdd', 'pickup_cluster', 'hour'], how = 'left')
df_morning.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,quarter_hour,service_obj_service_name,time_bucket,weekday,order_status,cancel_reason,second,minute,date,count,drop_cluster,num_of_orders,yyyymmdd,hour,pickup_cluster,Online_Captains,demand_to_supply_ratio


In [23]:
#Create a column for weekdays
df_morning['weekday_name'] = df_morning['weekday'].apply(lambda x: calendar.day_name[(x-1)])
df_evening['weekday_name'] = df_evening['weekday'].apply(lambda x: calendar.day_name[(x-1)])
df_morning.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,quarter_hour,service_obj_service_name,time_bucket,weekday,order_status,cancel_reason,second,minute,date,count,drop_cluster,num_of_orders,yyyymmdd,hour,pickup_cluster,Online_Captains,demand_to_supply_ratio,weekday_name


In [24]:
num_of_hex_morning = df_morning.groupby(['pickup_cluster'])['pickup_location_hex_8'].nunique().reset_index()
num_of_hex_evening = df_evening.groupby(['pickup_cluster'])['pickup_location_hex_8'].nunique().reset_index()
num_of_hex_morning = num_of_hex_morning.rename(columns = {'pickup_location_hex_8':'num_of_hexes'})
num_of_hex_evening = num_of_hex_evening.rename(columns = {'pickup_location_hex_8':'num_of_hexes'})

df_morning = df_morning.merge(num_of_hex_evening, on = ['pickup_cluster'], how = 'left')
df_evening = df_evening.merge(num_of_hex_evening, on = ['pickup_cluster'], how = 'left')
df_morning['online_captains_per_hex'] = round(df_morning['Online_Captains']/df_morning['num_of_hexes'], 4)
df_evening['online_captains_per_hex'] = round(df_evening['Online_Captains']/df_evening['num_of_hexes'], 4)

In [25]:
df_morning_filter = df_morning[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'Online_Captains', 'demand_to_supply_ratio', 'weekday_name', 'online_captains_per_hex']]
df_evening_filter = df_evening[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'Online_Captains', 'demand_to_supply_ratio', 'weekday_name', 'online_captains_per_hex']]

In [26]:
df_evening_filter.to_csv('chennai_demand_supply_evening.csv', index = False)